In [2]:
import tensorflow as tf
import time
import numpy as np
import os
import threading

In [ ]:
def load_data(path):
    if path == 'network':
        mnist = tf.keras.datasets.mnist
        return mnist.load_data()
    else:
        # path = 'file\\mnist.npz'
        f = np.load(path)
        x_train, y_train = f['x_train'], f['y_train']
        x_test, y_test = f['x_test'], f['y_test']
        f.close()
        return (x_train, y_train), (x_test, y_test)
    
(x_train, y_train), (x_test, y_test) = load_data('file\\mnist.npz')
x_train, x_test = x_train / 255.0, x_test / 255.0

In [3]:
def gen():
    print('  generator initiated')
    idx = 0
    while True:
        yield x_train[:32], y_train[:32]
        print('  wf>>>>>>>>>>>>>>>>>>>>>generator yielded a batch %d  PID:%d  ident:%d' % (idx, os.getpid(), threading.currentThread().ident))
        idx += 1
        time.sleep(3)
 
if __name__ == "__main__":
    tr_gen = gen()

    model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10, activation='softmax')
    ])
 
    print('  wf>>>>>>>>>>>>>>>>>>>>> PID:%d  ident:%d' % (os.getpid(), threading.currentThread().ident))
    model.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])
 
    start_time = time.time()
    model.fit_generator(generator=tr_gen, steps_per_epoch=20, max_queue_size=10)
    print('Total used time: %d '% (time.time() - start_time))

  wf>>>>>>>>>>>>>>>>>>>>> PID:62480  ident:56376
Instructions for updating:
Please use Model.fit, which supports generators.
  generator initiated
  ...
    to  
  ['...']
Train for 20 steps
  wf>>>>>>>>>>>>>>>>>>>>>generator yielded a batch 0  PID:62480  ident:47528
 4/20 [=====>........................] - ETA: 39s - loss: 2.1036 - accuracy: 0.3125
  wf>>>>>>>>>>>>>>>>>>>>>generator yielded a batch 4  PID:62480  ident:5817
 6/20 [========>.....................] - ETA: 36s - loss: 1.9333 - accuracy: 0.4740  wf>>>>>>>>>>>>>>>>>>>>>generator yielded a batch 5  PID:62480  ident:5817
  wf>>>>>>>>>>>>>>>>>>>>>generator yielded a batch 6  PID:62480  ident:70604
 8/20 [===========>..................] - ETA: 32s - loss: 1.7945 - accuracy: 0.5508  wf>>>>>>>>>>>>>>>>>>>>>generator yielded a batch 7  PID:62480  ident:4752
  wf>>>>>>>>>>>>>>>>>>>>>generator yielded a batch 8  PID:62480  ident:7060
10/20 [==============>...............] - ETA: 27s - loss: 1.6676 - accuracy: 0.6187  wf>>>>>>>>>>>>>>

In [1]:
class MNISTSequence(tf.keras.utils.Sequence):
    def __init__(self, x_set, y_set, batch_size):
        self.x, self.y = x_set, y_set
        self.batch_size = batch_size
        
    def __len__(self):        
        return int(np.ceil(len(self.x) / float(self.batch_size)))
    def __getitem__(self, idx):
        batch_x = self.x[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_y = self.y[idx * self.batch_size:(idx + 1) * self.batch_size]
        print('  wf>>>>>>>>>>>>>>>>>>>>>generator yielded a batch %d  PID:%d  ident:%d' % (idx, os.getpid(), threading.currentThread().ident))
        time.sleep(3)
        return np.array(batch_x), np.array(batch_y)
    
if __name__ == "__main__":
    tr_gen = MNISTSequence(x_train, y_train, 32)
    
    model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10, activation='softmax')
    ])
    print('  wf>>>>>>>>>>>>>>>>>>>>> PID:%d  ident:%d' % (os.getpid(), threading.currentThread().ident))
    model.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])
    start_time = time.time()
    model.fit_generator(generator=tr_gen, steps_per_epoch=20, max_queue_size=10, use_multiprocessing=True, workers=0)
    print('Total used time: %d '% (time.time() - start_time))


  wf>>>>>>>>>>>>>>>>>>>>> PID:56548  ident:70232
Instructions for updating:
Please use Model.fit, which supports generators.
  wf>>>>>>>>>>>>>>>>>>>>>generator yielded a batch 0  PID:56548  ident:70232
  ...
    to  
  ['...']
Train for 20 steps
  wf>>>>>>>>>>>>>>>>>>>>>generator yielded a batch 1740  PID:56548  ident:56836
  wf>>>>>>>>>>>>>>>>>>>>>generator yielded a batch 1682  PID:56548  ident:56836
 3/20 [===>..........................] - ETA: 53s - loss: 2.2717 - accuracy: 0.1042   wf>>>>>>>>>>>>>>>>>>>>>generator yielded a batch 1622  PID:56548  ident:1112
  wf>>>>>>>>>>>>>>>>>>>>>generator yielded a batch 1803  PID:56548  ident:63468
 5/20 [======>.......................] - ETA: 46s - loss: 2.2200 - accuracy: 0.1562  wf>>>>>>>>>>>>>>>>>>>>>generator yielded a batch 844  PID:56548  ident:5683
  wf>>>>>>>>>>>>>>>>>>>>>generator yielded a batch 1116  PID:56548  ident:56836
 9/20 [============>.................] - ETA: 33s - loss: 2.1190 - accuracy: 0.2535  wf>>>>>>>>>>>>>>>>>>>>>ge

在我自己的PC上worker只能为0，否则会报错。但在我用的GPU服务器测试是可以运行的，并能减少大量时间。